# Prepare data set

1. Load data set
2. 20%/ 80% split, 80% for cross-validation, 20% for training the meta learner (later)
3. Scale values 
4. Convert to tensoflow data set 

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
df = pd.read_pickle('classification_dataset.pkl')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace = True)
df

In [7]:
df['fps_blend_comp'] = (np.vstack(df.fps_blend_comp) - 1).tolist()

In [8]:
df['fps_blend_comp'] = (np.around(np.vstack(df.fps_blend_comp),decimals=5 )).tolist()

In [9]:
df = df[df['prop'] != 'Td']

In [ ]:
df['miscibility'] = np.array(df.val_1.values == df.val_2.values).astype(int)

In [11]:
df.insert(0, 'selector', pd.get_dummies(df.prop).values.tolist())


In [12]:
df

,selector,fps_blend_comp,bdid,c1,prop,val_1,val_2,miscibility
0,"[1, 0]","[0.20375, 0.85296, 0.12903, 0.27517, 0.25, 0.4...",547,0.80,Tg,238.15,238.15,1
1,"[1, 0]","[0.26403, 0.66891, 0.07692, 0.19685, 0.14286, ...",547,0.50,Tg,248.15,248.15,1
2,"[1, 0]","[0.2232, 0.78726, 0.11111, 0.24795, 0.21212, 0...",547,0.70,Tg,249.15,249.15,1
3,"[1, 0]","[0.26403, 0.66891, 0.07692, 0.19685, 0.14286, ...",547,0.50,Tg,338.15,338.15,1
4,"[1, 0]","[0.20375, 0.85296, 0.12903, 0.27517, 0.25, 0.4...",547,0.80,Tg,342.15,342.15,1
...,...,...,...,...,...,...,...,...
4244,"[0, 1]","[0.24404, 0.5775, 0.0, 0.07748, 0.0, 0.7198, 0...",2151,0.28,Tm,421.75,421.75,1
4245,"[0, 1]","[0.18245, 0.58736, 0.0, 0.23073, 0.0, 0.5371, ...",2151,0.73,Tm,422.65,422.65,1
4246,"[0, 1]","[0.20499, 0.58362, 0.0, 0.16797, 0.0, 0.60137,...",2151,0.56,Tm,422.75,422.75,1
4247,"[0, 1]","[0.21712, 0.60635, 0.0, 0.14943, 0.00689, 0.63...",2152,0.52,Tm,424.25,424.25,1


In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold

import tensorflow as tf
tf.random.set_seed(123)





# df['value'] = df.apply(lambda x: list([x['val_1'],
#                                         x['val_2']]),axis=1)    

df = df.sample(frac=1, random_state=123)
target_property_scaler = MinMaxScaler

batch_size = 200

# df['scaled_value'] = df['value']
skf = StratifiedKFold(n_splits=5)

props = df.prop.unique().tolist()

datasets = []
def get_dataset(df):
    fps_blends = np.stack(df.fps_blend_comp.values).astype(np.float32)

    selector = np.stack(df.selector.values).astype(np.float32)
    target = df.miscibility.values.astype(int)[:, np.newaxis]
    bdid = np.stack(df.bdid.values).astype(np.str)[:, np.newaxis]
    # miscibility = df.miscibility.values.astype(int)[:, np.newaxis]
    c1 = df.c1.values.astype(np.float32)[:, np.newaxis]
    # type_poly = np.stack(df.type.values).astype(np.str)[:, np.newaxis]
    dataset = tf.data.Dataset.from_tensor_slices(({'selector': selector, 'fps_blends': fps_blends, 'prop': df.prop, 'c1': c1, 'bdid':bdid}, target))


    dataset = dataset.cache().batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset


training_df, test_df = train_test_split(df, test_size=0.2, stratify=df.prop, random_state=123)
training_df, test_df = training_df.copy(), test_df.copy()
# print(props)
# print(skf.split(training_df, training_df.prop))
for train_index, val_index in skf.split(training_df, training_df.prop):
    
    train_df = training_df.iloc[train_index].copy()
    val_df = training_df.iloc[val_index].copy()
    datasets.append({'train': get_dataset(train_df), 'val': get_dataset(val_df)})

    
datasets_final = {'train': get_dataset(training_df), 'test': get_dataset(test_df)}



In [14]:
datasets_final

{'train': <PrefetchDataset element_spec=({'selector': TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), 'fps_blends': TensorSpec(shape=(None, 872), dtype=tf.float32, name=None), 'prop': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'c1': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'bdid': TensorSpec(shape=(None, 1), dtype=tf.string, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>,
 'test': <PrefetchDataset element_spec=({'selector': TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), 'fps_blends': TensorSpec(shape=(None, 872), dtype=tf.float32, name=None), 'prop': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'c1': TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), 'bdid': TensorSpec(shape=(None, 1), dtype=tf.string, name=None)}, TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>}

# Define the model

In [ ]:
import tensorflow.keras as tfk
import tensorflow as tf
from datetime import datetime 
from tensorflow.python.keras.engine import data_adapter
from keras_tuner import HyperModel
from keras_tuner.tuners import Hyperband, RandomSearch
import tensorflow_addons as tfa
from tensorflow import keras

class PropertyDonwsteam(tfk.Model):
    def __init__(self, hp):
        super().__init__()
        self.my_layers = []
        self.concat_at = hp.Int('concat_at', 0, 2)
        
        for i in range(hp.Int('num_layers', 3, 3)): 
            new_step = [               
            tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=352,
                                            max_value=544,
                                            step=64),),
            
            tf.keras.layers.PReLU(),
            tf.keras.layers.Dropout(hp.Float(
                'dropout_' + str(i),
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.1,
            )),
            ]

            self.my_layers.append(new_step)
        self.my_layers.append([tf.keras.layers.Dense(1, activation = 'sigmoid')])

    def call(self, inputs):
        x = inputs['fps_blends']

        for num, layer_step in enumerate(self.my_layers):
            if self.concat_at == num:
                x = tf.concat((x, inputs['selector']), -1)

            for layer in layer_step:
                x = layer(x)
    
        return x
    
    def predict_step(self, data):
        data = data_adapter.expand_1d(data)
        x, _, _ = data_adapter.unpack_x_y_sample_weight(data)
        # drop prop here
        prop = x['prop']
        del x['prop']
        return self(x, training=False), data[-1], prop

    
def build_model(hp):
    model = PropertyDonwsteam(hp)
    opt = tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-3]))
    opt = tfa.optimizers.SWA(opt)

    model.compile(
        optimizer=opt,
        loss=keras.losses.BinaryCrossentropy(from_logits=False),)
    return model

# Hyperparameter optimization of all of the five cross-validattion models

In [ ]:
import IPython

# import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

results, property_metric, best_values= [], [], []

prec_Tg = []
recall_Tg = []
accuracy_Tg = []
prec_Tm = []
recall_Tm = []
accuracy_Tm = []
for num, data in enumerate(datasets):
    
    tuner = Hyperband(
        build_model,
        objective='val_loss',
        max_epochs=300,
        seed=10,
        directory=f'hyperparameter_search_final_classification',
        project_name='fold_' + str(num)
        )

    reduce_lr = tfk.callbacks.ReduceLROnPlateau(
        factor=0.8,
        monitor="val_loss",
        verbose=1,
    )
    
    earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=40)
    
    class ClearTrainingOutput(tf.keras.callbacks.Callback):
        def on_train_end(*args, **kwargs):
            IPython.display.clear_output(wait = True)
    
    # Create an instance of the model
    tuner.search(data['train'],
                epochs=300,
                validation_data=data['val'],
                callbacks=[earlystop, reduce_lr, ClearTrainingOutput()],
                verbose=0
                )
    
    best_values.append(tuner.get_best_hyperparameters()[0].values)
    
    # Post processing
    best_model = tuner.get_best_models(1)[0]
    res = np.concatenate(best_model.predict(data['val']), -1)
    
    best_model.save(f'models/cv_final_class/{num}', include_optimizer=False)

    _df = pd.DataFrame(res, columns=['pred', 'target', 'prop'])
    _df['prop'] = _df.prop.apply(lambda x: x.decode('utf-8'))
    props = _df.prop.unique()
   
    res_Tg = _df[_df['prop'] == 'Tg']
    res_Tm = _df[_df['prop'] == 'Tm']      

    y_true = np.array(res_Tm.target.values, dtype = int)
    y_scores = res_Tm.pred.values
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_scores, pos_label = 1)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    predicted_value = []
    for i in res_Tm.pred.values:
        if i > optimal_threshold:
            predicted_value.append(1)
        else:
            predicted_value.append(0)
    m = tf.keras.metrics.Precision()
    m.update_state(np.array(res_Tm.target.values, dtype = int), predicted_value)
    prec_Tm.append(m.result().numpy())
    m = tf.keras.metrics.Recall()
    m.update_state(np.array(res_Tm.target.values, dtype = int), predicted_value)
    recall_Tm.append(m.result().numpy())
    m = tf.keras.metrics.Accuracy()
    m.update_state(np.array(res_Tm.target.values, dtype = int), predicted_value)
    accuracy_Tm.append(m.result().numpy())

    y_true = np.array(res_Tg.target.values, dtype = int)
    y_scores = res_Tg.pred.values
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_scores, pos_label = 1)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    predicted_value = []
    for i in res_Tg.pred.values:
        if i > optimal_threshold:
            predicted_value.append(1)
        else:
            predicted_value.append(0)
    m = tf.keras.metrics.Precision()
    m.update_state(np.array(res_Tg.target.values, dtype = int), predicted_value)
    prec_Tg.append(m.result().numpy())
    m = tf.keras.metrics.Recall()
    m.update_state(np.array(res_Tg.target.values, dtype = int), predicted_value)
    recall_Tg.append(m.result().numpy())
    m = tf.keras.metrics.Accuracy()
    m.update_state(np.array(res_Tg.target.values, dtype = int), predicted_value)
    accuracy_Tg.append(m.result().numpy())



In [34]:
df_performance = pd.DataFrame({'prec_Tg' :prec_Tg, 'recall_Tg' : recall_Tg, 'accuracy_Tg' : accuracy_Tg, 'prec_Tm' :prec_Tm, 'recall_Tm' : recall_Tm, 'accuracy_Tm' : accuracy_Tm }, columns=['prec_Tg','recall_Tg', 'accuracy_Tg', 'prec_Tm','recall_Tm', 'accuracy_Tm'])

In [35]:
df_performance['f1_score_Tg'] = 2*df_performance.prec_Tg*df_performance.recall_Tg/(df_performance.prec_Tg + df_performance.recall_Tg)
df_performance['f1_score_Tm'] = 2*df_performance.prec_Tm*df_performance.recall_Tm/(df_performance.prec_Tm + df_performance.recall_Tm)


In [36]:
df_performance

,prec_Tg,recall_Tg,accuracy_Tg,prec_Tm,recall_Tm,accuracy_Tm,f1_score_Tg,f1_score_Tm
0,0.965015,0.837975,0.838641,0.969466,0.798742,0.827751,0.897019,0.875862
1,0.944751,0.863636,0.842887,0.952000,0.734568,0.765550,0.902375,0.829268
2,0.980000,0.736842,0.764331,0.973451,0.696203,0.755981,0.841202,0.811808
3,0.967456,0.799511,0.802548,0.959016,0.717791,0.755981,0.875502,0.821053
4,0.948718,0.753181,0.759574,0.949580,0.729032,0.770335,0.839716,0.824818


In [37]:
df_performance.mean(axis=0)

prec_Tg        0.961188
recall_Tg      0.798229
accuracy_Tg    0.801596
prec_Tm        0.960703
recall_Tm      0.735267
accuracy_Tm    0.775120
f1_score_Tg    0.871163
f1_score_Tm    0.832562
dtype: float32